In [1]:
import csv
import os
from datetime import datetime

# Global variables
expenses = []
monthly_budget = 0.0
DATA_DIR = "data/"
CSV_FILE = os.path.join(DATA_DIR, "expense-tracker.csv")
BUDGET_FILE = os.path.join(DATA_DIR, "budget.csv")

def create_data_directory():
    """Create data directory if it doesn't exist"""
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)
        print(f"Created data directory: {DATA_DIR}\n")

def create_csv_if_not_exists():
    """Create CSV files if they don't exist"""
    # Create expense tracker file
    if not os.path.exists(CSV_FILE):
        try:
            with open(CSV_FILE, 'w', newline='') as file:
                fieldnames = ['date', 'category', 'amount', 'description']
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
            print(f"Created new expense tracker file: {CSV_FILE}\n")
        except Exception as e:
            print(f"Error creating CSV file: {e}\n")
    
    # Create budget file
    if not os.path.exists(BUDGET_FILE):
        try:
            with open(BUDGET_FILE, 'w', newline='') as file:
                fieldnames = ['monthly_budget']
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
            print(f"Created new budget file: {BUDGET_FILE}\n")
        except Exception as e:
            print(f"Error creating budget file: {e}\n")

def load_budget():
    """Load monthly budget from file"""
    global monthly_budget
    if os.path.exists(BUDGET_FILE):
        try:
            with open(BUDGET_FILE, 'r', newline='') as file:
                reader = csv.DictReader(file)
                rows = list(reader)
                if rows and 'monthly_budget' in rows[0]:
                    monthly_budget = float(rows[0]['monthly_budget'])
                    return True
        except Exception as e:
            print(f"Error loading budget: {e}\n")
    return False

def save_budget():
    """Save monthly budget to file"""
    try:
        with open(BUDGET_FILE, 'w', newline='') as file:
            fieldnames = ['monthly_budget']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerow({'monthly_budget': monthly_budget})
    except Exception as e:
        print(f"Error saving budget: {e}\n")

def load_expenses():
    """Load expenses from CSV file at program start"""
    global expenses
    if os.path.exists(CSV_FILE):
        try:
            with open(CSV_FILE, 'r', newline='') as file:
                reader = csv.DictReader(file)
                expenses = []
                for row in reader:
                    # Convert amount to float
                    row['amount'] = float(row['amount'])
                    expenses.append(row)
            if expenses:
                print(f"Loaded {len(expenses)} expenses from file.\n")
        except Exception as e:
            print(f"Error loading expenses: {e}\n")

def is_first_time_user():
    """Check if this is the first time user is running the application"""
    return not os.path.exists(CSV_FILE) or os.path.getsize(CSV_FILE) <= 50  # Header only

def validate_date(date_str):
    """Validate date format YYYY-MM-DD"""
    try:
        datetime.strptime(date_str, '%Y-%m-%d')
        return True
    except ValueError:
        return False

def add_expense():
    """Add a new expense with validation"""
    print("\n--- Add New Expense ---")
    
    # Get and validate date
    while True:
        date = input("Enter date (YYYY-MM-DD): ").strip()
        if validate_date(date):
            break
        else:
            print("Error: Invalid date format. Please use YYYY-MM-DD format.")
    
    # Get and validate category
    while True:
        category = input("Enter category (max 15 characters): ").strip()
        if len(category) == 0:
            print("Error: Category cannot be empty.")
        elif len(category) > 15:
            print("Error: Category exceeds 15 characters limit.")
        else:
            break
    
    # Get and validate amount
    while True:
        try:
            amount = float(input("Enter amount: ").strip())
            if amount <= 0:
                print("Error: Amount must be greater than 0.")
            else:
                break
        except ValueError:
            print("Error: Please enter a valid number for amount.")
    
    # Get and validate description
    while True:
        description = input("Enter description (max 150 characters): ").strip()
        if len(description) == 0:
            print("Error: Description cannot be empty.")
        elif len(description) > 150:
            print("Error: Description exceeds 150 characters limit.")
        else:
            break
    
    # Create expense dictionary
    expense = {
        'date': date,
        'category': category,
        'amount': amount,
        'description': description
    }
    
    expenses.append(expense)
    print("\nExpense added successfully!")

def view_expenses():
    """Display all expenses with validation"""
    print("\n--- View All Expenses ---")
    
    if not expenses:
        print("No expenses recorded yet.")
        return
    
    print(f"\nTotal Expenses: {len(expenses)}\n")
    print(f"{'Date':<12} {'Category':<15} {'Amount':<10} {'Description':<50}")
    print("-" * 90)
    
    for expense in expenses:
        # Validate that all required fields exist
        if all(key in expense for key in ['date', 'category', 'amount', 'description']):
            try:
                date = expense['date']
                category = expense['category']
                amount = float(expense['amount'])
                description = expense['description']
                
                print(f"{date:<12} {category:<15} ${amount:<9.2f} {description:<50}")
            except (ValueError, KeyError):
                print("Skipping incomplete entry...")
        else:
            print("Skipping incomplete entry (missing required fields)...")

def set_budget():
    """Set monthly budget"""
    global monthly_budget
    print("\n--- Set Monthly Budget ---")
    
    while True:
        try:
            budget = float(input("Enter your monthly budget: $").strip())
            if budget <= 0:
                print("Error: Budget must be greater than 0.")
            else:
                monthly_budget = budget
                save_budget()
                print(f"\nMonthly budget set to ${monthly_budget:.2f}")
                break
        except ValueError:
            print("Error: Please enter a valid number.")

def track_budget():
    """Track budget and display status"""
    global monthly_budget
    
    print("\n--- Track Budget ---")
    
    if monthly_budget == 0:
        print("No budget set. Please set a budget first.")
        set_budget()
        return
    
    # Calculate total expenses
    total_expenses = sum(float(expense['amount']) for expense in expenses)
    
    print(f"\nMonthly Budget: ${monthly_budget:.2f}")
    print(f"Total Expenses: ${total_expenses:.2f}")
    
    if total_expenses > monthly_budget:
        overspent = total_expenses - monthly_budget
        print(f"\n⚠️  WARNING: You have exceeded your budget by ${overspent:.2f}!")
    else:
        remaining = monthly_budget - total_expenses
        print(f"\n✓ You have ${remaining:.2f} left for the month.")

def save_expenses():
    """Save expenses to CSV file"""
    try:
        with open(CSV_FILE, 'w', newline='') as file:
            if expenses:
                fieldnames = ['date', 'category', 'amount', 'description']
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(expenses)
        print("\n✓ Expenses saved successfully!")
    except Exception as e:
        print(f"\nError saving expenses: {e}")

def welcome_first_time_user():
    """Welcome message and budget setup for first-time users or users without budget"""
    print("\n" + "="*60)
    print("     🎉 WELCOME TO PERSONAL EXPENSE TRACKER! 🎉")
    print("="*60)
    
    if monthly_budget == 0:
        print("\nTo get started, let's set up your monthly budget goal!")
        print("This will help you track your spending and stay within your limits.")
    else:
        print("\nIt looks like this is your first time using the tracker.")
        print("Let's get you started by setting up your monthly budget goal!")
        print("This will help you track your spending and stay within your limits.")
    
    print("-"*60)
    
    set_budget()
    
    print("\n" + "="*60)
    print("Great! You're all set. Let's start tracking your expenses!")
    print("="*60)

def check_existing_budget():
    """Check if budget is already set and offer to edit or continue"""
    if monthly_budget > 0:
        print("\n" + "="*60)
        print(f"Your monthly budget goal is already set: ${monthly_budget:.2f}")
        print("="*60)
        print("\n1. Edit monthly budget")
        print("2. Continue with existing budget")
        print("-"*60)
        
        while True:
            choice = input("\nEnter your choice (1-2): ").strip()
            if choice == '1':
                set_budget()
                break
            elif choice == '2':
                print("\nContinuing with existing budget...\n")
                break
            else:
                print("Invalid choice. Please enter 1 or 2.")

def display_menu():
    """Display interactive menu"""
    print("\n" + "="*50)
    print("       PERSONAL EXPENSE TRACKER")
    print("="*50)
    print("1. Add expense")
    print("2. View expenses")
    print("3. Track budget")
    print("4. Save expenses")
    print("5. Exit")
    print("="*50)

def main():
    """Main program loop"""
    # Create data directory if it doesn't exist
    create_data_directory()
    
    # Create CSV files if they don't exist
    create_csv_if_not_exists()
    
    # Load budget from file
    budget_loaded = load_budget()
    
    # Load expenses at startup
    load_expenses()
    
    # Check if first-time user or budget not set
    first_time = is_first_time_user()
    
    if first_time and not budget_loaded:
        # Brand new user
        welcome_first_time_user()
    elif not budget_loaded or monthly_budget == 0:
        # User has some data but no budget
        welcome_first_time_user()
    else:
        # Returning user with existing budget
        print("\n*** Welcome Back to Personal Expense Tracker ***\n")
        check_existing_budget()
    
    while True:
        display_menu()
        
        choice = input("\nEnter your choice (1-5): ").strip()
        
        if choice == '1':
            add_expense()
        elif choice == '2':
            view_expenses()
        elif choice == '3':
            track_budget()
        elif choice == '4':
            save_expenses()
        elif choice == '5':
            print("\nSaving expenses before exit...")
            save_expenses()
            save_budget()
            print("\nThank you for using Personal Expense Tracker!")
            print("Goodbye!\n")
            break
        else:
            print("\nInvalid choice. Please enter a number between 1 and 5.")

if __name__ == "__main__":
    main()

Loaded 2 expenses from file.


*** Welcome Back to Personal Expense Tracker ***


Your monthly budget goal is already set: $400.00

1. Edit monthly budget
2. Continue with existing budget
------------------------------------------------------------



Continuing with existing budget...


       PERSONAL EXPENSE TRACKER
1. Add expense
2. View expenses
3. Track budget
4. Save expenses
5. Exit

Invalid choice. Please enter a number between 1 and 5.

       PERSONAL EXPENSE TRACKER
1. Add expense
2. View expenses
3. Track budget
4. Save expenses
5. Exit

Saving expenses before exit...

✓ Expenses saved successfully!

Thank you for using Personal Expense Tracker!
Goodbye!

